In [1]:
import numpy as np
import pandas as pd
import healpy as hp
import matplotlib.pyplot as plt

hecateFile = pd.read_csv('/home/db/Desktop/AstroProject/Data/hecate_v1.1_for_GW.csv')
fitsFile = '/home/db/Desktop/AstroProject/Data/MCMC_TF2_LowSpin_AllSky.fits'

[prob, distmu, distsigma, distnorm], header = hp.read_map(fitsFile, field=[0, 1, 2, 3], h = True) # creates 2 arrays, one with 4 columns, and one for the header column that is needed to find an abstract value which is the NSIDE for the ang2pix function

headerDict = {nameKeys:value for nameKeys,value in header} # Creates a dictionnary between a key and a value associated

HECATE_pixelsIndex = hp.ang2pix(headerDict['NSIDE'], hecateFile['RA'], np.array(hecateFile['DEC']), lonlat=True) # Converts the 2 angle values we have to pixel values, that we'll then make correspond with the FITS pixels

In [2]:
# P2D top 5 sources

P2D = prob[HECATE_pixelsIndex] # P2D probability associated to all galaxy of HECATE

sumProb = np.sum(P2D)

P2D_norm = P2D/sumProb # Normalized P2D array

fiveMaxP2DIndex = np.argsort(P2D_norm)[-5:][::-1] # takes the 5 largest elements sorted from largest to smallest ([-5:] is from smallest to largest only)

fiveMaxP2D = P2D_norm[fiveMaxP2DIndex] # fiveMaxProb is the top 5 galaxy candidates that can be the source of the GW

fiveMaxP2DNames = hecateFile['OBJNAME'][fiveMaxP2DIndex]

fiveMaxP2DNames = fiveMaxP2DNames.to_numpy()

print(np.c_[fiveMaxP2DNames, fiveMaxP2D])

[['PGC169672' 0.006675970091842589]
 ['PGC831458' 0.00656682202026065]
 ['PGC158506' 0.006526787685675716]
 ['PGC837800' 0.0063802623433190314]
 ['2MASXJ13042517-2042527' 0.0062625413769155265]]


In [3]:
# Variables

P_pixel = P2D # Renaming the non-normalized P2D for all galaxies

N_pixel = distnorm[HECATE_pixelsIndex]

D_galaxy = hecateFile['D'].to_numpy()

mu_pixel = distmu[HECATE_pixelsIndex]

mu_pixel = np.array([x if x >= 0 else 0 for x in mu_pixel]) # changes all negative values of mu_pixel to 0

sig_pixel = distsigma[HECATE_pixelsIndex]

exponent = -1/2*((D_galaxy-mu_pixel)/sig_pixel)**2

GaussFactor = np.exp(exponent) # Gaussian exponential of (1) of Ducoin paper

In [4]:
# P3D top 5 sources

P3D = P_pixel*N_pixel*GaussFactor # Formula (1) of the Ducoin paper

P3D = np.nan_to_num(P3D) # sets all nan values to 0

sumP3D = np.sum(P3D)

P3D_norm = P3D/sumP3D # Normalized P3D array

fiveMaxP3DIndex = np.argsort(P3D_norm)[-5:][::-1] 

fiveMaxP3D = P3D_norm[fiveMaxP3DIndex]

fiveMaxP3DNames = hecateFile['OBJNAME'][fiveMaxP3DIndex]

fiveMaxP3DNames = fiveMaxP3DNames.to_numpy()

print(np.c_[fiveMaxP3DNames, fiveMaxP3D])

[['ESO508-004' 0.0964901982774173]
 ['ESO575-053' 0.08491642944762096]
 ['NGC4993' 0.08356755221946364]
 ['ESO575-055' 0.08168905970345927]
 ['ESO508-024' 0.06740727980139116]]


In [5]:
# P3D*n(M*) top 5 sources

Smass = hecateFile['logM_HEC'].to_numpy() # Column logM_HEC from the csv file

Smass = np.nan_to_num(Smass)

PnM = P3D*np.exp(Smass) # Using formula (1) of the Artale paper

PnM = np.nan_to_num(PnM)

sumPnM = np.sum(PnM)

PnM = PnM/sumPnM # Normalized PnM array

fiveMaxPnMIndex = np.argsort(PnM)[-5:][::-1]

fiveMaxPnM = PnM[fiveMaxPnMIndex]

fiveMaxPnMnames = hecateFile['OBJNAME'][fiveMaxPnMIndex]

fiveMaxPnMnames = fiveMaxPnMnames.to_numpy()

print(np.c_[fiveMaxPnMnames, fiveMaxPnM])

[['NGC4993' 0.2749864562253773]
 ['ESO575-053' 0.11071220901540405]
 ['ESO508-024' 0.10329815898779292]
 ['ESO575-055' 0.08281312950348892]
 ['NGC4968' 0.07316961900854449]]


In [57]:
# Copying all logSFR_W values of the Wise file to a copied logSFR_HEC column of the hecate file

SFR = (hecateFile['logSFR_HEC'].to_numpy())

SFR1 = SFR

# wiseFile = pd.read_csv('/home/db/Desktop/AstroProject/Data/candidates_WISE_SFRs.csv')

# wiseFile = wiseFile.to_numpy()

wiseFile = np.genfromtxt('/home/db/Desktop/AstroProject/Data/candidates_WISE_SFRs.csv', delimiter = "," , names = True)

SFR2 = (wiseFile['logSFR_W'])

def fill_SFR(old, new):
    return new if not np.isfinite(old) else old

# Other way of writing fill_SFR would be :
# def fill_SFR(old, new):
    # if np.isfinite(old)
        # return old
    # return new
    
#pgcWise = wiseFile[0]['pgc']
#print(pgcWise)

for wise_row in wiseFile : # le 1er terme dans la nomenclature "for" désigne l'index de ligne du tableau en question
    #print(wise_row) #ok
    target_PGC = wise_row['pgc']
    target_logSFR = wise_row['logSFR_W']
    #print(target_PGC) #ok
    indexHEC = np.where(hecateFile['OBJNAME'] == target_PGC)[0]
    
    #type(indexHEC) returns "tuple"
    #type(indexHEC[0]) returns "numpy.ndarray"
    
    #type(indexHEC)
    #print(indexHEC)
    #indexHEC = np.where(hecateFile['OBJNAME'] == target_PGC)[0][0]
    #print(indexHEC)
    #print(hecateFile['logSFR_HEC'].iloc[indexHEC])
    SFR1[indexHEC] = fill_SFR(hecateFile['logSFR_HEC'][indexHEC], target_logSFR)
    

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [53]:
type(indexHEC)

tuple

In [ ]:
# P3D*n(M*,SFR) top 5 sources

SFR = (hecateFile['logSFR_HEC'].to_numpy())

SFR = np.nan_to_num(SFR)

PnMSFR = P3D*np.exp(Smass+SFR) # Using formula (2) of the Artale paper

PnMSFR = np.nan_to_num(PnMSFR)

sumPnMSFR = np.sum(PnMSFR)

PnMSFR = PnMSFR/sumPnMSFR # Normalized PnMSFR array

fiveMaxPnMSFRIndex = np.argsort(PnMSFR)[-5:][::-1]

fiveMaxPnMSFR = PnMSFR[fiveMaxPnMSFRIndex]

fiveMaxPnMSFRnames = hecateFile['OBJNAME'][fiveMaxPnMSFRIndex]

fiveMaxPnMSFRnames = fiveMaxPnMSFRnames.to_numpy()

print(np.c_[fiveMaxPnMSFRnames, fiveMaxPnMSFR])

In [ ]:
# P3D*n(M*,SFR, Z) top 5 sources

Z = (hecateFile['METAL'].to_numpy())

Z = np.nan_to_num(Z)

#minMaxZ = [np.min(Z), np.max(Z)]

#print(minMaxZ)

PnAll = P3D*np.exp(Smass+SFR+Z) # Using formula (9) of the K. paper

PnAll = np.nan_to_num(PnAll)

sumPnAll = np.sum(PnAll)

PnAll = PnAll/sumPnAll # Normalized PnAll array

fiveMaxPnAllIndex = np.argsort(PnAll)[-5:][::-1]

fiveMaxPnAll = PnAll[fiveMaxPnAllIndex]

fiveMaxPnAllnames = hecateFile['OBJNAME'][fiveMaxPnAllIndex]

fiveMaxPnAllnames = fiveMaxPnAllnames.to_numpy()

print(np.c_[fiveMaxPnAllnames, fiveMaxPnAll])

# IMPORTANT : The results here seem to indicate that 1) none of the 5 top galaxies have metallicity 

# and 2) that known metallicity data for other galaxies isn't enough for them to have priority over those 5

In [ ]:
# Function that applies n(M*), n(M*,SFR) or n(M*,SFR, Z) according to data


# Plan

# Ca peut pas être 3 fonctions, car certaines galaxies peuvent avoir SFR mais pas Smass

# En fait au final, si on enlève les NaN de Smass, SFR et Z, la fonction PnAll = P3D*np.exp(Smass+SFR+Z) fait ça très bien, comme ça somme sur 0

# La fonction doit pouvoir filtrer

In [ ]:
# Results

print("P2D", np.c_[fiveMaxP2DNames, fiveMaxP2D], " "
      , "P3D", np.c_[fiveMaxP3DNames, fiveMaxP3D], " "
      , "P3D*n(M*)", np.c_[fiveMaxPnMnames, fiveMaxPnM], " "
      , "P3D*n(M*,SFR)", np.c_[fiveMaxPnMSFRnames, fiveMaxPnMSFR], " "
      , "P3D*n(M*,SFR, Z)", np.c_[fiveMaxPnAllnames, fiveMaxPnAll],sep='\n')